## Постановка задачи

* Дообучить GPT по русскоязычным текстам новостей писать заголовки к ним.

Для файнтюнинга возьмем среднюю модель `sberbank-ai/rugpt3medium_based_on_gpt2`, чтобы она уместилась на GPU.

Также укажем библиотеке pytorch, что вычисления мы будем проводить на графическом процессоре с поддержкой `cuda`.

В качестве датасета возьмем `IlyaGusev/gazeta` — он компактный, есть поля `text` и `title`, то есть идеально для обучения модели писать заголовки.

## Установка


In [2]:
import torch
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling

DEVICE = torch.device("cuda:0")
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"

## Модель и токенайзер

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(DEVICE)

if tokenizer.eos_token is None:
    tokenizer.add_special_tokens({'eos_token': ''})
    model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
model.generation_config.pad_token_id = tokenizer.pad_token_id

## Датасет

In [ ]:
dataset = load_dataset("IlyaGusev/gazeta")

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 60964
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6369
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6793
    })
})


## Преобразование для обучения

In [ ]:
MAX_LEN = 512
MAX_NEW = 32
SRC_MAX = MAX_LEN - MAX_NEW

def preprocess(batch):
    prompts = [f"Новость: {t}\nЗаголовок:" for t in batch["text"]]
    titles  = [" " + y + tokenizer.eos_token for y in batch["title"]]

    enc_p = tokenizer(prompts, truncation=True, max_length=SRC_MAX,
                      padding=False, add_special_tokens=False)
    enc_t = tokenizer(titles,  truncation=True, max_length=MAX_NEW,
                      padding=False, add_special_tokens=False)

    input_ids, attention_mask, labels = [], [], []
    for p_ids, t_ids in zip(enc_p["input_ids"], enc_t["input_ids"]):
        ids = p_ids + t_ids
        am  = [1]*len(ids)
        lab = [-100]*len(p_ids) + t_ids

        pad = MAX_LEN - len(ids)
        if pad > 0:
            ids += [tokenizer.pad_token_id]*pad
            am  += [0]*pad
            lab += [-100]*pad
        else:
            ids, am, lab = ids[:MAX_LEN], am[:MAX_LEN], lab[:MAX_LEN]

        input_ids.append(ids)
        attention_mask.append(am)
        labels.append(lab)

    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [ ]:
tokenized = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/60964 [00:00<?, ? examples/s]

Map:   0%|          | 0/6369 [00:00<?, ? examples/s]

Map:   0%|          | 0/6793 [00:00<?, ? examples/s]

In [ ]:
# Примеры исходных данных
print("=== Примеры ===")
for i in range(3):
    print("Новость:", dataset["train"][i]["text"][:200], "...")
    print("Заголовок:", dataset["train"][i]["title"])
    print()

=== Примеры ===
Новость: Сегодня транспортный налог начисляется в зависимости от мощности автомобиля, причем цена для «сильных» машин выше, чем для малолитражек. Также ставку налога могут корректировать региональные власти: с ...
Заголовок: Налог в бак

Новость: Словосочетание «музыкальный кинофестиваль» уже не звучит непривычным оксюмороном — в наше бедное на гениев время разобраться в современной музыке порой проще с помощью наблюдения за участниками процес ...
Заголовок: Секс, наркотики и темный зал

Новость: После более чем 12-часовых консультаций Совет Безопасности ООН согласовал заявление председателя по израильскому штурму «Флотилии свободы». Совбез осудил захват флотилии, требует немедленно отпустить  ...
Заголовок: Осудить и отпустить



## Обучение

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import os
os.environ["WANDB_DISABLED"] = "true"

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    fp16=True,
    learning_rate=5e-5,
    report_to="none",
    seed=42,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"].select(range(20000)),   # 20k примеров
    eval_dataset=tokenized["validation"].select(range(1000)),
    processing_class=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,2.802200,2.823937
2,2.605600,2.827636


TrainOutput(global_step=10000, training_loss=2.702637417602539, metrics={'train_runtime': 3982.2427, 'train_samples_per_second': 10.045, 'train_steps_per_second': 2.511, 'total_flos': 3.714802778112e+16, 'train_loss': 2.702637417602539, 'epoch': 2.0})

## Инференс

In [ ]:
import re, torch
torch.manual_seed(42)                 # детерминируем
tokenizer.padding_side = "left"       # для batched decoder-only стабильнее

def lead(text, max_sent=3, max_chars=600):
    sents = re.split(r'(?<=[.!?…])\s+', text.strip())
    return ' '.join(sents[:max_sent])[:max_chars]

def batch_generate(texts, refs=None, max_new_tokens=24, mode="beam"):
    # 1) укоротим новость до первых 2–3 предложений
    texts2 = [lead(t) for t in texts]

    # 2) сформируем промпт и явно передадим attention_mask
    prompts = [f"Новость: {t}\nЗаголовок:" for t in texts2]
    enc = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    enc = {k: v.to(DEVICE) for k, v in enc.items()}

    # 3) без сэмплирования: beam-search делает заголовки фактичнее
    out = model.generate(
        **enc,
        max_new_tokens=max_new_tokens,
        num_beams=5,
        do_sample=False,
        length_penalty=0.9,
        no_repeat_ngram_size=2,
        eos_token_id=tokenizer.eos_token_id,
    )

    dec = tokenizer.batch_decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    rows = []
    for i, (p, full) in enumerate(zip(prompts, dec)):
        j = full.rfind("Заголовок:")
        pred = full[j+len("Заголовок:"):].strip().split("\n")[0].strip(" -—:\"'`") if j != -1 else full.strip()
        item = {"news": texts2[i], "pred": pred}
        if refs is not None:
            item["ref"] = refs[i]
        rows.append(item)
    return rows


In [1]:
sample = dataset["test"].select(range(5))
rows = batch_generate(sample["text"], refs=sample["title"], mode="sample")

for r in rows:
    print("НОВОСТЬ:", r["news"][:400], "...\nPRED :", r["pred"])
    if "ref" in r: print("REF  :", r["ref"])
    print()

NameError: name 'dataset' is not defined

## Общий вывод

Мы дообучили модель **RuGPT3-medium** на корпусе новостей *IlyaGusev/gazeta*, чтобы она умела генерировать заголовки по текстам статей. Несмотря на ограниченное количество данных и небольшое число эпох, модель научилась улавливать основную тему и формировать краткие заголовки, иногда совпадающие с эталонными по смыслу.